In [19]:
### HEADER ##
# description: use autoencoder to reduce dimensionality of sequence embeddings
# author: HR

import os
import pandas as pd
import numpy as np

os.system('pwd')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

import keras.backend.tensorflow_backend as K
K.set_session

<function keras.backend.tensorflow_backend.set_session(session)>

In [20]:
### INPUT ###

# seq2vec + TF-IDF
seq2vec = pd.read_csv('../RUNS/HumanProteome/word2vec_model/hp_sequence_repres_w5_d100_seq2vec-TFIDF.csv')

meta = seq2vec.iloc[:, 0:3]
emb = np.array(seq2vec.iloc[:, 3:103], dtype = 'float64')


In [43]:
### MAIN PART ###

# model

def build_and_compile_AE():
    
    tf.keras.backend.clear_session()
    
    inp = layers.Input(shape = (100, ), name = 'input')
    encoded = layers.Dense(32, activation = 'tanh', name = 'encoded')(inp)
    decoded = layers.Dense(100, activation = 'sigmoid', name = 'decoded')(encoded)
    
    autoencoder = keras.Model(inputs = inp, outputs = decoded)
    
    autoencoder.compile(optimizer = keras.optimizers.Adadelta(),
                       loss = keras.losses.BinaryCrossentropy(),
                       metrics = ['accuracy'])
    
    return autoencoder

AE = build_and_compile_AE()
AE.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 100)               0         
_________________________________________________________________
encoded (Dense)              (None, 32)                3232      
_________________________________________________________________
decoded (Dense)              (None, 100)               3300      
Total params: 6,532
Trainable params: 6,532
Non-trainable params: 0
_________________________________________________________________


In [44]:
# split data
X_train, X_test = train_test_split(emb, test_size = .2)

X_train.shape

(33888, 100)

In [45]:
# fit model

fit = AE.fit(X_train, X_train,
            epochs = 100,
            batch_size = 64,
            shuffle = True,
            verbose = 1,
            validation_data = (X_test, X_test))

AE.predict(X_test, X_test)

Train on 33888 samples, validate on 8473 samples
Epoch 1/100
33888/33888 [==============================] - 1s 29us/sample - loss: -0.6671 - acc: 0.7723 - val_loss: -1.2076 - val_acc: 0.9000
Epoch 2/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.2468 - acc: 0.8444 - val_loss: -1.3061 - val_acc: 0.8446
Epoch 3/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.3420 - acc: 0.8700 - val_loss: -1.4226 - val_acc: 0.8849
Epoch 4/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.4460 - acc: 0.8934 - val_loss: -1.4924 - val_acc: 0.8946
Epoch 5/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.4872 - acc: 0.8998 - val_loss: -1.5126 - val_acc: 0.8984
Epoch 6/100
33888/33888 [==============================] - 1s 24us/sample - loss: -1.4979 - acc: 0.9009 - val_loss: -1.5186 - val_acc: 0.8989
Epoch 7/100
33888/33888 [==============================] - 1s 24us/sample - loss: -1.5031 - acc: 0.

33888/33888 [==============================] - 1s 24us/sample - loss: -1.5314 - acc: 0.8705 - val_loss: -1.5468 - val_acc: 0.8722
Epoch 59/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.5314 - acc: 0.8702 - val_loss: -1.5468 - val_acc: 0.8691
Epoch 60/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.5315 - acc: 0.8691 - val_loss: -1.5469 - val_acc: 0.8691
Epoch 61/100
33888/33888 [==============================] - 1s 23us/sample - loss: -1.5316 - acc: 0.8691 - val_loss: -1.5470 - val_acc: 0.8685
Epoch 62/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.5317 - acc: 0.8679 - val_loss: -1.5470 - val_acc: 0.8682
Epoch 63/100
33888/33888 [==============================] - 1s 23us/sample - loss: -1.5318 - acc: 0.8680 - val_loss: -1.5471 - val_acc: 0.8691
Epoch 64/100
33888/33888 [==============================] - 1s 25us/sample - loss: -1.5319 - acc: 0.8687 - val_loss: -1.5471 - val_acc: 0.8683
Epoch 65/100

TypeError: only size-1 arrays can be converted to Python scalars

In [37]:
# save weights and metrics
AE.save_weights('AE/seq2vec-TFIDF.h5')

val = []
name = list(fit.history.keys())
for i, elem in enumerate(fit.history.keys()):
    val.append(fit.history[elem])

m = list(zip(name, val))
m = pd.DataFrame(m)
pd.DataFrame.to_csv(m, 'AE/metrics_ext_seq2vec-TFIDF.csv', header=False, index = False)
